In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity




In [2]:
df=pd.read_csv('spotify_millsongdata.csv')

In [28]:
df.head()

,artist,song,text
0,The White Stripes,little cream soda,"well, every highway that i go down \r\nseems ..."
1,Enrique Iglesias,stay here tonight,i know it's late and you're tired \r\nand we'...
2,Nick Cave,knockin' on joe,"these chains of sorrow, they are heavy, it is ..."
3,Dan Fogelberg,along the road,joy at the start \r\nfear in the journey \r\...
4,Who,dangerous,can you feel it? watching you in the darkness ...


In [3]:
df=df.sample(20000).drop('link', axis=1).reset_index(drop=True)


In [4]:
df.groupby('artist').count()

,song,text
artist,,
'n Sync,30,30
ABBA,36,36
Ace Of Base,26,26
Adam Sandler,21,21
Adele,23,23
...,...,...
Zoegirl,12,12
Zornik,3,3
Zox,7,7


NOW that we have assigned the link to another column we can drop them


In [5]:
df.shape

(20000, 3)

In [6]:
artist_list=df.groupby('artist').count()

Now we are having 643 artist list in our dataset


In [7]:
artist_list.count()

song    634
text    634
dtype: int64

In [8]:
df['song']=df['song'].str.lower()

In [9]:

p=df['text'].str.count('\n').sum()

In [10]:
df['text'] = df['text'].apply(lambda x: str(x).lower().replace(r'^\w\s',' ').replace(r'\n',' ',))


In [11]:
df.columns

Index(['artist', 'song', 'text'], dtype='object')

In [12]:

# import nltk
# it is a nlp library , it is used to process the text data
# from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

In this the similar words which are having same meaning but differ in spelling are changed into one common word


In [13]:
def token(text):
    token=nltk.word_tokenize(text)
    a=[stemmer.stem(word) for word in token]
    return " ".join(a)
    

In [14]:
# # import nltk
# # nltk.download('punkt')
# token(df['text'][0])
# token('hello world beautiful beauty')

In [15]:
df['text'].apply(lambda x:token(x))

0        well , everi highway that i go down seem to be...
1        i know it 's late and you 're tire and we 'd b...
2        these chain of sorrow , they are heavi , it is...
3        joy at the start fear in the journey joy in th...
4        can you feel it ? watch you in the dark can yo...
                               ...                        
19995    in a long and involv convers with myself i saw...
19996    [ origin by the door ] well , the clock say it...
19997    monday 's eye have got stuck on the ceil again...
19998    did you hear a stori about cocain lil ? she ha...
19999    make me feel someth i use to feel i use to fee...
Name: text, Length: 20000, dtype: object

In [16]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity



tdif=TfidfVectorizer(analyzer='word', stop_words='english')

In [17]:
matrix=tdif.fit_transform(df['text'])

In [18]:
# from sklearn.metrics.pairwise import cosine_similarity
# from scipy.sparse import csr_matrix

# # Assuming 'matrix' is a sparse matrix in CSR format
# sparse_matrix = csr_matrix(matrix)

# # Compute cosine similarity using sparse matrices
similar = cosine_similarity(matrix)

In [19]:
similar[0]

array([1.        , 0.09492742, 0.0133813 , ..., 0.02531565, 0.01865388,
       0.02075045])

In [20]:
df['song']

0             little cream soda
1             stay here tonight
2               knockin' on joe
3                along the road
4                     dangerous
                  ...          
19995         carlisle wheeling
19996              soul kitchen
19997          just another day
19998                   cocaine
19999    make me feel something
Name: song, Length: 20000, dtype: object

In [21]:
df[df['song']=="city's burning"].index[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

Now we are making recommder function for getting the songs


In [22]:
def recommder(song_name):
    idx=df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similar[idx])), key=lambda x: x[1], reverse=True)
    song=[]
    for s_id in distance[1:20]:
        song.append(df.iloc[s_id[0]].song)
    return song
    
    

In [ ]:
recommder("city's burning")

["baby i'm burning",
 'both ends burning',
 'burning ground',
 'fires',
 'kiss of life',
 'dixie fire',
 'burning bridges',
 "london's burning",
 'slow burning love',
 "burnin' up",
 'dear john',
 'funny how it fades away',
 'rain of fire',
 'big city nights',
 'burning flag',
 'no face no name no number',
 'tiger in a burning tree',
 'burning down the house',
 'keep the candle burning']

Now we are gonna do the dumping of our code to bytes through pickle with dump method

In [23]:
import pickle

In [27]:
pickle.dump(similar, open('similar.pkl', 'wb'))

In [26]:
pickle.dump(df, open('df.pkl', 'wb'))